<a href="https://colab.research.google.com/github/alanbseo/MFAworkshop/blob/main/MFA_2_MaterialIntensity_EmbeddedCarbon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages
import os
import matplotlib.pyplot as plt

try:
  import geopandas as gpd
except:
  !pip install geopandas
  import geopandas as gpd

# import rasterio pacakge 
try:
  import rasterio
except:
  !pip install rasterio
  import rasterio

# import transform and features
from rasterio import transform
from rasterio import features
from rasterio.enums import Resampling

from google.colab import drive


# Mount Gdrive 

drive.mount('/content/gdrive/')

# set working directory
default_path = '/content/gdrive/MyDrive/Net zero built environment/MFA workshop'
 
os.chdir(default_path) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.1 MB/s eta 0:00:00
Mounted at /content/gdrive/


In [ ]:
# Read the rasterised Volume and Code data 
volume_out_fn = './images/volume_100m.tif'
code_out_fn = './images/code_100m.tif'


with rasterio.open(volume_out_fn) as src:
    print(src.profile)
    volume_img = src.read(1)  


with rasterio.open(code_out_fn) as src:
    print(src.profile)
    code_img = src.read(1) 

  

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 67, 'height': 75, 'count': 1, 'crs': CRS.from_epsg(5179), 'transform': Affine(100.0, 0.0, 951300.0,
       0.0, -100.0, 1959400.0), 'blockysize': 15, 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 67, 'height': 75, 'count': 1, 'crs': CRS.from_epsg(5179), 'transform': Affine(100.0, 0.0, 951300.0,
       0.0, -100.0, 1959400.0), 'blockysize': 75, 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
